In [1]:
from dotenv import load_dotenv
import os
load_dotenv()
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")

In [2]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_community.vectorstores import Chroma, FAISS
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_google_genai import GoogleGenerativeAI
from langchain.chains import LLMMathChain, LLMChain
from langchain.prompts import PromptTemplate

d:\SOURAV\GenAI\HotelChatBot\env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
llm =  GoogleGenerativeAI(model="gemini-pro", google_api_key=GOOGLE_API_KEY)
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001", google_api_key=GOOGLE_API_KEY)

In [4]:
loader=PyPDFLoader("knowledge.pdf")
docs = loader.load_and_split()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1024)
documents = text_splitter.split_documents(docs)

In [5]:
PERSIST_STORAGE = "./storage"
chroma_db = Chroma.from_documents(documents=documents, embedding= embeddings, persist_directory=PERSIST_STORAGE)


# faiss_db = FAISS.from_documents(documents=documents, embedding= embeddings)
# faiss_db.save_local("faiss_index")
# faiss_db=FAISS.load_local("faiss_index", embeddings=embeddings, allow_dangerous_deserialization=True)

In [6]:
# retriever = faiss_db.as_retriever()
retriever = chroma_db.as_retriever()

In [7]:
from langchain.tools.retriever import create_retriever_tool
retriever_tool = create_retriever_tool(
    retriever=retriever,
    name="hotel_search",
    description="Search for information about Hotel check-in, check out, types of rooms, price of rooms. \
        For any questions on the above topics, you must use this tool!",
)

In [8]:
# TAVILY_API_KEY

In [9]:
# search = TavilySearchResults()

In [10]:
# search.dict()

In [11]:
from langchain.agents import Tool
problem_chain = LLMMathChain.from_llm(llm=llm)
math_tool = Tool.from_function(name="Calculator",
                func=problem_chain.run,
                 description="Useful for when you need to answer questions \
about math. This tool is only for math questions and nothing else for the price calculation of hotel rooms.")

In [12]:
tools = [retriever_tool,math_tool]

# Agent

In [13]:
from langchain.agents import create_react_agent, AgentExecutor
from langchain import hub

In [14]:
prompt = hub.pull("hwchase17/react")

In [15]:
print(prompt.template)

Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}


In [16]:
agent = create_react_agent(llm=llm, tools=tools, prompt=prompt)

In [17]:
agent_executer = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [18]:
agent_executer.invoke({"input":"What are the types of rooms in the hotel?"})



> Entering new AgentExecutor chain...
Action: hotel_search
Action Input: What types of rooms are available?4. The number of rooms available for each room type:  
• Standard Room: 60 rooms available  
• Deluxe Room: 30 rooms available  
• Executive Suite: 10 suites available  
This distribution ensures that we can accommodate various preferences and group sizes while 
maintaining a balance between room types.  
 
5. The accommodation capacity for each room type:  
• Standard Room: Accommodates up to 2 adults.  
• Deluxe Room: Accommodates up to 3 adults or 2 adults and 1 child (with an additional rollaway 
bed, subject to availability and extra charges).  
• Executive Suite: Accommodates up to 4 adults or 2 adults and 2 children (with an additional sofa 
bed, subject to availability and extra charges).  
These capacities are designed to provide comfort and flexibility for different types of guests, whether 
traveling alone, as a couple, or with family.  
 
6. Price of Each Type of Roo

{'input': 'What are the types of rooms in the hotel?',
 'output': 'The types of rooms available are:\n• Standard Room: Comfortable and cozy, suitable for solo travelers or couples.\n• Deluxe Room: Spacious and elegantly furnished, ideal for families or guests seeking added comfort.\n• Executive Suite: Luxurious accommodations with separate living and sleeping areas, perfect for business travelers or those seeking a premium experience.'}

In [19]:
agent_executer.invoke({"input":"What will be the price for booking 3 Executive suites?"})



> Entering new AgentExecutor chain...
Action: hotel_search
Action Input: Find the price of booking an Executive suite• Deluxe Room: ₹10,500 per night  
• Executive Suite: ₹17,500 per night  
Please note that prices are subject to change based on seasonal demand and availability.  
 
7. Water Availability:  
Complimentary bottled water is provided in each room upon arrival.  
Additionally, our hotel ensures round -the-clock availability of fresh, potable water throughout the 
premises, adhering to stringent quality standards for guest satisfaction and convenience.  
 
General Policies:  
1. Reservation and Cancellation:  
Reservations can be made online through our website, over the phone, or in person at the front desk.  
Cancellation or modification of reservations must be made at least 24 hours prior to the scheduled arrival 
to avoid cancellation charges, unless otherwise stated during booking.  
 
 
2. Smoking Policy:  
For the comfort and safety of all guests, smoking is strictl

{'input': 'What will be the price for booking 3 Executive suites?',
 'output': '52500'}

In [20]:
agent_executer.invoke({"input":"How many rooms are available?"})



> Entering new AgentExecutor chain...
Action: hotel_search
Action Input: Show me all the roomsHotel Policy and Services Overview  
 
1. Number of Rooms Available:  
Our hotel boasts a total of 100 rooms, distributed across various categories to accommodate the diverse 
needs and preferences of our guests.  
 
2. Check -In and Checkout Timings:  
• Check -In Time: 2:00 PM  
• Checkout Time: 12:00 PM  
We understand the importance of flexibility; therefore, early check -in and late checkout requests are 
subject to availability and may incur additional charges.  
 
3. Types of Rooms:  
• Standard Room: Comfortable and cozy, suitable for solo travellers  or couples.  
• Deluxe Room: Spacious and elegantly furnished, ideal for families or guests seeking added 
comfort.  
• Executive Suite: Luxurious accommodations with separate living and sleeping areas, perfect for 
business travelers or those seeking a premium experience.  
 
4. The number of rooms available for each room type:  
• Sta

{'input': 'How many rooms are available?',
 'output': 'There are 100 rooms available in the hotel'}